## UGCA Group Assignment 3

# Group Members
1. Chetna Singhal
2. David Kinman
3. Prajval Gupta
4. Subhayu Chakravarty
5. Whitt Hyde

In [2]:
#from selenium import webdriver
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy

import en_core_web_lg

### PART A - Extracing Beer Reviews

In [4]:
### Scraping beer reviewa via Selenium ###

driver = webdriver.Chrome(executable_path='/Users/livestrong19/Downloads/chromedriver')
driver.get('https://www.beeradvocate.com/beer/top-rated/')

### Fetch names of 250 beers and URL link
beer_dict={}
for i in range(2,252):
    beer_path=driver.find_element_by_xpath('//div[@id="ba-content"]/table/tbody/tr['+str(i)+']/td[2]/a')
    beer_name=driver.find_element_by_xpath('//div[@id="ba-content"]/table/tbody/tr['+str(i)+']/td[2]/a/b').text
    beer_dict[beer_name]=beer_path.get_attribute('href')
driver.close()

In [5]:
# Extraction Round 1
### Fetch first 25 reviews for each of the 250 beers

beer_ratings = pd.DataFrame(columns = ['Product Name','User Rating','Product Review']) 

try:
    for key,value in beer_dict.items():
        driver = webdriver.Chrome('/Users/livestrong19/Downloads/chromedriver')
        #driver = webdriver.Chrome('/Users/prajvalgupta/Downloads/chromedriver 2')
        driver.get(value)
        reviews = driver.find_elements_by_xpath('//div[@id="rating_fullview_content_2"]')
        for review in reviews:
            review_text = review.text
            rating = review.find_element_by_xpath('.//span[1]').text
            extra1 = review.find_element_by_xpath('.//span[2]').text
            extra2 = review.find_element_by_xpath('.//span[3]').text
            extra3 = review.find_element_by_xpath('.//span[4]').text
            extra4 = review.find_element_by_xpath('.//span[5]').text
            extra5 = review.find_element_by_xpath('.//div').text
            
            if extra1:
                review_text = review_text.replace(extra1,"")
            if extra2:
                review_text = review_text.replace(extra2,"")
            if extra3:
                review_text = review_text.replace(extra3,"")
            if extra4:
                review_text = review_text.replace(extra4,"")
            if extra5:
                review_text = review_text.replace(extra5,"")

            beer_ratings.loc[len(beer_ratings)] = [key,rating,review_text]

        driver.close()
except:
    pass

In [8]:
beer_ratings

,Product Name,User Rating,Product Review
0,Kentucky Brunch Brand Stout,4.8,4.8 rDev \n\n\n2016 Silver Wax. Aroma has whi...
1,Kentucky Brunch Brand Stout,4.74,4.74 rDev \n\n\nThe beer pours Pitch Black wi...
2,Kentucky Brunch Brand Stout,4.68,4.68 rDev \n\n\nProbably the smoothest beer I...
3,Kentucky Brunch Brand Stout,5,"5 rDev \n\n\nDark black, very thick, a little..."
4,Kentucky Brunch Brand Stout,4.97,4.97 rDev \n\n\nPoured black as ink with thin...
...,...,...,...
4826,Black & Wild,3.44,3.44 rDev \n\n\nNot anything I want to try ag...
4827,Black & Wild,4.66,4.66 rDev \n\n\n8oz pour on draft at the brew...
4828,Black & Wild,4.38,4.38 rDev \n\n\nWas surprised that I got to t...
4829,Black & Wild,4.74,4.74 rDev \n\n\nOne of the most unique wild's...


In [60]:
beer_ratings['Product Name'].value_counts()

Double Stack                                           25
Parabola                                               25
Barrel Aged Imperial German Chocolate Cupcake Stout    25
Clover                                                 25
Nectarine Premiere                                     25
                                                       ..
4th Anniversary                                        22
Anabasis                                               20
It Was All A Dream                                     20
Chemtrailmix                                           17
Black & Wild                                            5
Name: Product Name, Length: 195, dtype: int64

In [19]:
beer1 = list(beer_dict.keys())
beer2 = list(beer_ratings['Product Name'].unique())
print(len(beer1),len(beer2))
beer_missing = list(set(beer1)-set(beer2))

In [53]:
# Extraction Round 2
### Fetch reviews for missing 55 beers

beer_ratings2 = pd.DataFrame(columns = ['Product Name','User Rating','Product Review']) 


for key,value in beer_dict.items():
    if key in beer_missing:
        driver = webdriver.Chrome('/Users/livestrong19/Downloads/chromedriver')
        driver.get(value+"?view=beer&sort=&start=25")
        reviews = driver.find_elements_by_xpath('//div[@id="rating_fullview_content_2"]')
        
        for review in reviews:
            review_text = review.text
            
            rating = review.find_element_by_xpath('.//span[1]').text
            extra1 = review.find_element_by_xpath('.//span[2]').text
            extra2 = review.find_element_by_xpath('.//span[3]').text
            extra3 = review.find_element_by_xpath('.//span[4]').text
            try:
                extra4 = review.find_element_by_xpath('.//span[5]').text
                if extra4:
                    review_text = review_text.replace(extra4,"")
            except:
                pass

            extra5 = review.find_element_by_xpath('.//div').text

            if extra1:
                review_text = review_text.replace(extra1,"")
            if extra2:
                review_text = review_text.replace(extra2,"")
            if extra3:
                review_text = review_text.replace(extra3,"")

            if extra5:
                review_text = review_text.replace(extra5,"")
            
            beer_ratings2.loc[len(beer_ratings2)] = [key,rating,review_text]

        driver.close()

In [54]:
beer_ratings2

,Product Name,User Rating,Product Review
0,Ten FIDY - Bourbon Barrel Aged,4.38,4.38 rDev \n\n\n19.2oz can into a goblet. Pac...
1,Ten FIDY - Bourbon Barrel Aged,4,"4 rDev \n\n\nVintage optics - oil black, quar..."
2,Ten FIDY - Bourbon Barrel Aged,4.58,4.58 rDev \n\n\nPours almost black with a kha...
3,Ten FIDY - Bourbon Barrel Aged,4.37,4.37 rDev \n\n\n2017 Vintage GVBX Draught 112...
4,Ten FIDY - Bourbon Barrel Aged,4.43,"4.43 rDev \n\n\nOn tap at Beer Shop, NYC.\nPo..."
...,...,...,...
1163,Sure Bet,4.74,"4.74 rDev \n\n\nBomber. Pours opaque orange, ..."
1164,Sure Bet,4.07,"4.07 rDev \n\n\nNot their best, but another s..."
1165,Sure Bet,4.48,4.48 rDev \n\n\nLooks fantastic - hazy orange...
1166,Sure Bet,4.46,"4.46 rDev \n\n\nL - foggy copper orange, nice..."


In [66]:
beer_ratings_final = beer_ratings.append(beer_ratings2, ignore_index = True)
beer_ratings_final.to_csv("beer_reviews.csv")

### PART B - Selecting 3 beer attributes using word frequency analysis

In [3]:
# Load beer reviews into a dataframe
df = pd.read_csv("beer_reviews.csv",usecols=["Product Name","User Rating","Product Review"])

# Remove newline from each post & drop rows with null values
df = df.replace('\n','', regex=True)
df = df.replace('rDev','', regex=True)
df = df.dropna()
df

,Product Name,User Rating,Product Review
0,Kentucky Brunch Brand Stout,4.80,"4.8 2016 Silver Wax. Aroma has whiskey, mapl..."
1,Kentucky Brunch Brand Stout,4.74,4.74 The beer pours Pitch Black with a froth...
2,Kentucky Brunch Brand Stout,4.68,4.68 Probably the smoothest beer I have ever...
3,Kentucky Brunch Brand Stout,5.00,"5 Dark black, very thick, a little bit of ta..."
4,Kentucky Brunch Brand Stout,4.97,4.97 Poured black as ink with thin ruby edge...
5,Kentucky Brunch Brand Stout,4.75,4.75 she’s got heat....but man is she someth...
6,Kentucky Brunch Brand Stout,5.00,5 Amazing brew. The maple aroma pours out of...
7,Kentucky Brunch Brand Stout,4.30,4.3 Finally got to try the white whale. Pour...
8,Kentucky Brunch Brand Stout,4.85,4.85 Had this on tap at the 12-15-18 release...
9,Kentucky Brunch Brand Stout,4.89,4.89 Had this on tap At the KBBS release 12/...


In [4]:
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

def review_cleaner(text):
    
    # Remove numbers
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    
    lower_case = letters_only.lower()
    
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [5]:
clean_reviews = []

for i in range(df.shape[0]):
    clean_reviews.append(review_cleaner(df['Product Review'][i]))
    
clean_reviews = pd.DataFrame(clean_reviews)
df["Clean Product Review"]=clean_reviews
df

,Product Name,User Rating,Product Review,Clean Product Review
0,Kentucky Brunch Brand Stout,4.80,"4.8 2016 Silver Wax. Aroma has whiskey, mapl...",silver wax aroma has whiskey maple toffee and ...
1,Kentucky Brunch Brand Stout,4.74,4.74 The beer pours Pitch Black with a froth...,the beer pours pitch black with a frothy tan h...
2,Kentucky Brunch Brand Stout,4.68,4.68 Probably the smoothest beer I have ever...,probably the smoothest beer i have ever had sm...
3,Kentucky Brunch Brand Stout,5.00,"5 Dark black, very thick, a little bit of ta...",dark black very thick a little bit of tan head...
4,Kentucky Brunch Brand Stout,4.97,4.97 Poured black as ink with thin ruby edge...,poured black as ink with thin ruby edges at de...
5,Kentucky Brunch Brand Stout,4.75,4.75 she’s got heat....but man is she someth...,she s got heat but man is she somethin nice lo...
6,Kentucky Brunch Brand Stout,5.00,5 Amazing brew. The maple aroma pours out of...,amazing brew the maple aroma pours out of this...
7,Kentucky Brunch Brand Stout,4.30,4.3 Finally got to try the white whale. Pour...,finally got to try the white whale pours like ...
8,Kentucky Brunch Brand Stout,4.85,4.85 Had this on tap at the 12-15-18 release...,had this on tap at the release in decorah the ...
9,Kentucky Brunch Brand Stout,4.89,4.89 Had this on tap At the KBBS release 12/...,had this on tap at the kbbs release pour was s...


In [6]:
# Remove all punctuation characters from each tweet and convert text to lowercase for efficient frequency counting
# punctuation includes !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~

def remove_punctuations(item):
    for p in punctuation:
        item = item.strip().replace(p,'')
    return item

# Function to remove stopwords
def remove_stopwords(s):
    stop_words = set(stopwords.words('english'))
    return [w for w in s if not w in stop_words] 

In [7]:
# Tokenize the posts
df['Tokens'] = df["Clean Product Review"].apply(word_tokenize).apply(list)
# df['Tokens'] = df['Tokens'].apply(remove_punctuations)
df['Tokens'] = df['Tokens'].apply(remove_stopwords)
df

,Product Name,User Rating,Product Review,Clean Product Review,Tokens
0,Kentucky Brunch Brand Stout,4.80,"4.8 2016 Silver Wax. Aroma has whiskey, mapl...",silver wax aroma has whiskey maple toffee and ...,"[silver, wax, aroma, whiskey, maple, toffee, u..."
1,Kentucky Brunch Brand Stout,4.74,4.74 The beer pours Pitch Black with a froth...,the beer pours pitch black with a frothy tan h...,"[beer, pours, pitch, black, frothy, tan, head,..."
2,Kentucky Brunch Brand Stout,4.68,4.68 Probably the smoothest beer I have ever...,probably the smoothest beer i have ever had sm...,"[probably, smoothest, beer, ever, smelled, bet..."
3,Kentucky Brunch Brand Stout,5.00,"5 Dark black, very thick, a little bit of ta...",dark black very thick a little bit of tan head...,"[dark, black, thick, little, bit, tan, head, s..."
4,Kentucky Brunch Brand Stout,4.97,4.97 Poured black as ink with thin ruby edge...,poured black as ink with thin ruby edges at de...,"[poured, black, ink, thin, ruby, edges, degree..."
5,Kentucky Brunch Brand Stout,4.75,4.75 she’s got heat....but man is she someth...,she s got heat but man is she somethin nice lo...,"[got, heat, man, somethin, nice, lots, barrel,..."
6,Kentucky Brunch Brand Stout,5.00,5 Amazing brew. The maple aroma pours out of...,amazing brew the maple aroma pours out of this...,"[amazing, brew, maple, aroma, pours, bottle, u..."
7,Kentucky Brunch Brand Stout,4.30,4.3 Finally got to try the white whale. Pour...,finally got to try the white whale pours like ...,"[finally, got, try, white, whale, pours, like,..."
8,Kentucky Brunch Brand Stout,4.85,4.85 Had this on tap at the 12-15-18 release...,had this on tap at the release in decorah the ...,"[tap, release, decorah, aroma, amazing, step, ..."
9,Kentucky Brunch Brand Stout,4.89,4.89 Had this on tap At the KBBS release 12/...,had this on tap at the kbbs release pour was s...,"[tap, kbbs, release, pour, super, dark, head, ..."


In [8]:
count = []

# Count the frequency of the words
for i in range(len(df["Tokens"])):
    count += df.iloc[i]["Tokens"]

count = [x.lower() for x in count]
word_freq = nltk.FreqDist(count)

# 50 most frequent words
top_words = word_freq.most_common(500)
top_words

[('beer', 4981),
 ('head', 3472),
 ('taste', 2844),
 ('chocolate', 2610),
 ('dark', 2576),
 ('like', 2537),
 ('sweet', 2247),
 ('one', 2171),
 ('nice', 2082),
 ('coffee', 2079),
 ('bourbon', 2062),
 ('light', 1955),
 ('orange', 1906),
 ('well', 1860),
 ('good', 1849),
 ('nose', 1846),
 ('notes', 1825),
 ('vanilla', 1787),
 ('medium', 1723),
 ('finish', 1701),
 ('aroma', 1660),
 ('flavor', 1646),
 ('pours', 1636),
 ('carbonation', 1616),
 ('bit', 1611),
 ('fruit', 1591),
 ('white', 1579),
 ('body', 1507),
 ('bottle', 1504),
 ('little', 1497),
 ('citrus', 1460),
 ('great', 1453),
 ('black', 1450),
 ('really', 1446),
 ('barrel', 1423),
 ('mouthfeel', 1422),
 ('smooth', 1394),
 ('overall', 1343),
 ('flavors', 1340),
 ('thick', 1316),
 ('oak', 1298),
 ('lacing', 1258),
 ('glass', 1248),
 ('brown', 1224),
 ('color', 1193),
 ('feel', 1186),
 ('malt', 1120),
 ('creamy', 1091),
 ('smell', 1085),
 ('bitterness', 1063),
 ('poured', 1044),
 ('much', 1039),
 ('dry', 1008),
 ('sweetness', 1001),
 ('

In [9]:
word_list = list(sum(top_words, ()))
new_word_list = {word_list[i]: word_list[i + 1] for i in range(0, len(word_list), 2)} 
new_word_list

{'beer': 4981,
 'head': 3472,
 'taste': 2844,
 'chocolate': 2610,
 'dark': 2576,
 'like': 2537,
 'sweet': 2247,
 'one': 2171,
 'nice': 2082,
 'coffee': 2079,
 'bourbon': 2062,
 'light': 1955,
 'orange': 1906,
 'well': 1860,
 'good': 1849,
 'nose': 1846,
 'notes': 1825,
 'vanilla': 1787,
 'medium': 1723,
 'finish': 1701,
 'aroma': 1660,
 'flavor': 1646,
 'pours': 1636,
 'carbonation': 1616,
 'bit': 1611,
 'fruit': 1591,
 'white': 1579,
 'body': 1507,
 'bottle': 1504,
 'little': 1497,
 'citrus': 1460,
 'great': 1453,
 'black': 1450,
 'really': 1446,
 'barrel': 1423,
 'mouthfeel': 1422,
 'smooth': 1394,
 'overall': 1343,
 'flavors': 1340,
 'thick': 1316,
 'oak': 1298,
 'lacing': 1258,
 'glass': 1248,
 'brown': 1224,
 'color': 1193,
 'feel': 1186,
 'malt': 1120,
 'creamy': 1091,
 'smell': 1085,
 'bitterness': 1063,
 'poured': 1044,
 'much': 1039,
 'dry': 1008,
 'sweetness': 1001,
 'stout': 954,
 'hops': 940,
 'caramel': 938,
 'grapefruit': 915,
 'full': 905,
 'big': 878,
 'tropical': 868,


In [10]:
# Finding top 3 attributes basis word frequency analysis

beer_attribute_list = [["aggressive",0],["balanced",0],["complex",0],["crisp",0],["fruity",0],["hoppy",0],["malty",0],["robust",0]]
beer_attribute_list = pd.DataFrame(beer_attribute_list)
beer_attribute_list.columns = ["Attribute","Word_Frequency"]

for i in range(len(beer_attribute_list["Attribute"])):
    for key, value in new_word_list.items():
        if beer_attribute_list["Attribute"][i] in key:
            beer_attribute_list["Word_Frequency"].iloc[i] = value

beer_attribute_list = beer_attribute_list.sort_values(by=['Word_Frequency'],ascending=False)
beer_attribute_list

/Users/subhayuchakravarty/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Attribute,Word_Frequency
1,balanced,832
4,fruity,530
2,complex,312
3,crisp,297
5,hoppy,275
6,malty,193
0,aggressive,0
7,robust,0


In [11]:
# Storing top 3 attributes in file
beer_attribute_top3_list = beer_attribute_list.iloc[0:3,0]
beer_attribute_top3_list.to_csv("user_beer_attributes.csv",index=False, header="Attributes")

### PART C - Similarity analysis between Beer Attributes and Beer Reviews

In [12]:
attribute_list = pd.read_csv("user_beer_attributes.csv")
attribute_list

,Attribute
0,balanced
1,fruity
2,complex


In [13]:
nlp = en_core_web_lg.load()

In [14]:
listToStr = ' '.join([str(elem) for elem in attribute_list["Attribute"]]) 
doc1 = nlp(listToStr)

for i in range(len(df["Clean Product Review"])):
    doc2 = nlp(df["Clean Product Review"][i])
    df.loc[i,"Similarity Score"] = doc1.similarity(doc2)

In [15]:
df = df.sort_values(by='Similarity Score', ascending=False)
similarity_df = df[["Product Name","Clean Product Review","Similarity Score"]]
similarity_df.columns = ["Product Name","Product Review","Similarity Score"]
similarity_df.to_excel(r'Beer_Attribute_Review_Similarity.xlsx', index=False)

In [16]:
top_300_reviews = pd.read_excel(r'Beer_Attribute_Review_Similarity.xlsx',nrows=300)
top_300_reviews

,Product Name,Product Review,Similarity Score
0,Ephraim,bright refereshing peppery orange citrus piney...,0.785144
1,Galaxy Dry Hopped Fort Point Pale Ale,wow this is amazing incredible aromas of peach...,0.781336
2,Heady Topper,golden with slight amber hue good foamy head a...,0.779848
3,Duck Duck Gooze,on tap at beavertown extravaganza aroma has so...,0.773673
4,Saison Bernice,tasted from bottle classic farmhouse saison po...,0.767015
5,Pseudo Sue - Double Dry-Hopped,oz can from tavour hazy pale golden spotty hea...,0.762522
6,Double Galaxy,soft gold foamy head aroma has underripe melon...,0.760734
7,Flora Plum,insanely tart and funky bright acidity lemon w...,0.759532
8,Atrial Rubicite,incredible raspberry and cherry aromas with so...,0.756422
9,Sang Noir,projectno bottling date but bought in late app...,0.753732


### PART D - Sentiment Analysis of Beer Reviews

In [18]:
# pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score

In [19]:
for i in range(len(top_300_reviews["Product Review"])):
    top_300_reviews.loc[i,"Sentiment Score"] = sentiment_analyzer_scores(top_300_reviews["Product Review"][i])['compound']
    top_300_reviews = top_300_reviews.sort_values(by='Sentiment Score', ascending=False)

top_300_reviews

,Product Name,Product Review,Similarity Score,Sentiment Score
63,Darkness,ml bottle into snifter bottled on pours dense ...,0.727849,0.9976
196,Fyodor,th review ml bottle into snifter bottled in po...,0.710494,0.9964
276,Mocha Wednesday,ml bottle into snifter bottled on pours nearly...,0.704642,0.9959
194,Art,ml bottle batch dated march poured into a hill...,0.710759,0.9951
20,Double Citra®,got in a bif from roguer ml swing top growler ...,0.742694,0.9950
13,Montmorency Vs Balaton,ml bottle into signature tulip glass blend bot...,0.745879,0.9946
25,Double Dry Hopped Congress Street,ounce can into tulip glass canned on pours opa...,0.739988,0.9944
279,Tweak,ounce bottle into snifter bottled on pours den...,0.704462,0.9942
205,Notorious Triple IPA,had this on tap at the mule bar in winooski vt...,0.709592,0.9938
46,Saint Lamvinus,th review ml bottle into tulip glass bottled o...,0.732611,0.9931


### PART E - Top 3 beer recommendations - Method I

In [47]:
top_300_reviews=top_300_reviews.groupby('Product Name')['Similarity Score','Sentiment Score'].mean().sort_values(by = ['Similarity Score'],ascending=False)

top_300_reviews.head(3)

,Similarity Score,Sentiment Score
Product Name,,
Ephraim,0.785144,0.8958
Heady Topper,0.779848,0.7003
The Broken Truck,0.748612,0.6207


#### Initial Recommendations

Out of the top 300 similar review we took sorted the 148 beers by the average similarity. 
Based solely on the chart above the top 3 beers would be:

  **Beer, Similarity, Sentiment**<br>
1. **Ephraim**,   0.785144,   0.895800 <br>
2. **Heady Topper**,   0.779848,   0.700300 <br>
3. **The Broken Truck**,   0.748612,   0.620700<br>

In [49]:
top_300_reviews=top_300_reviews.groupby('Product Name')['Similarity Score','Sentiment Score'].mean().sort_values(by = ['Sentiment Score'],ascending=False)

top_300_reviews.head(3)

,Similarity Score,Sentiment Score
Product Name,,
Darkness,0.727849,0.9976
Fyodor,0.710494,0.9964
Mocha Wednesday,0.704642,0.9959


However we noticed that this may not be the best approach because some similarity differences are very small but the sentiment was vastly greater, thus a strictly algorithmic approach would not suffice. 
We decided to recommend beers slightly less similar, in order to recommend beers with higher sentiment.

Since we have the top similar beers in the 300 reviews out of the 6000, taking the sentiment scores is a way to get the best recommendations.

#### Final Recommendations - based on similarity and sentiment

The top beers as per their sentiment scores are as follows:

  **Beer, Similarity, Sentiment**<br>
1. **Darkness**,   0.727849, 0.9976 <br>
2. **Fyodor**,   0.710494,	0.9964 <br>
3. **Mocha Wednesday**,   0.704642,	0.9959<br>


<br>

### PART F - Top 3 beer recommendations - Method II

In [35]:
df_top=df.copy()
df_top.head()

,Product Name,User Rating,Product Review,Clean Product Review,Tokens,Similarity Score
1698,Ephraim,4.60,"4.6 Bright, refereshing, peppery, orange citr...",bright refereshing peppery orange citrus piney...,"[bright, refereshing, peppery, orange, citrus,...",0.785144
2063,Galaxy Dry Hopped Fort Point Pale Ale,4.58,"4.58 11/2014 - Wow, this is amazing; incredi...",wow this is amazing incredible aromas of peach...,"[wow, amazing, incredible, aromas, peach, dank...",0.781336
124,Heady Topper,4.25,4.25 Golden with slight amber hue. Good foam...,golden with slight amber hue good foamy head a...,"[golden, slight, amber, hue, good, foamy, head...",0.779848
732,Duck Duck Gooze,4.50,4.5 On tap at Beavertown Extravaganza. Aroma...,on tap at beavertown extravaganza aroma has so...,"[tap, beavertown, extravaganza, aroma, sweet, ...",0.773673
4104,Saison Bernice,4.44,4.44 Tasted from bottle. Classic farmhouse s...,tasted from bottle classic farmhouse saison po...,"[tasted, bottle, classic, farmhouse, saison, p...",0.767015


In [37]:
df_top.drop('Similarity Score', axis=1, inplace=True)
df_rating=df_top.groupby(['Product Name']).mean().reset_index().sort_values(['User Rating'], ascending=False)

df_rating.head(3)

,Product Name,User Rating
146,Kentucky Brunch Brand Stout,4.812000
52,Chemtrailmix,4.811176
24,Barrel-Aged Abraxas,4.798000


If we were to ignore similarity and sentiment scores of the beers and only recommend based on the ratings given by users, we would arrive at very different results.

#### Recommendations - based only on User Rating

The top beers as per user ratings are as follows:

  **Beer, User Rating**<br>
1. **Kentucky Brunch Brand Stout**,	4.812000 <br>
2. **Chemtrailmix**,	4.811176<br>
3. **Barrel-Aged Abraxas**,	4.798000<br>


In [78]:
# Similarity and Sentiment with User Rating:
joined_df=pd.merge(top_300_reviews, df_rating, on='Product Name', how='left')
top3_rated=joined_df[joined_df['Product Name'].isin(['Kentucky Brunch Brand Stout','Chemtrailmix','Barrel-Aged Abraxas'])]
top3_rated

,Product Name,Similarity Score,Sentiment Score,User Rating
37,Barrel-Aged Abraxas,0.716299,0.8979,4.798


In [79]:
df_sentiment=df[df['Product Name'].isin(['Kentucky Brunch Brand Stout','Chemtrailmix','Barrel-Aged Abraxas'])]
df_sentiment=df_sentiment.groupby('Product Name')['Similarity Score','Sentiment Score'].mean()
df_sentiment

,Similarity Score,Sentiment Score
Product Name,,
Barrel-Aged Abraxas,0.588432,0.827764
Chemtrailmix,0.585982,0.900206
Kentucky Brunch Brand Stout,0.571752,0.788880


We find here that two of the top rated beers were not even present in the top 300 similar reviews, based on the attributes we had considered.

In summary, we had taken the attribute vector as ['balanced','fruity','complex'].
Based on this we found 300 reviews that were most similar to the vector above and shortlisted 148 beers that were the most similar to these attributes.
On these 300 reviews we found the most favored beers by analysing the sentiment of these reviews.
Using the similarity scores and sentiment scores we have a recommendation of 3 beers which are the most similar to our entered characteristics.

1. **Darkness**<br>
2. **Fyodor**<br>
3. **Mocha Wednesday**<br>

This is vastly different from the recommended beers we get when only taking into account the user rating or popularity of beers on the whole dataset. 

1. **Kentucky Brunch Brand Stout**
2. **Chemtrailmix**
3. **Barrel-Aged Abraxas**

These three are not very similar to the attributes we had considered. We had similarity scores above 0.70 for the similar beers, whereas the top-rated beers have their similarity in the vicinity of 0.58. So, we can conclude that the top-rated beers are not really similar in their attributes to the beers we can recommend, considering similarity and positive sentiment.

So, our final three recommendations are **Darkness**, **Fyodor** and **Mocha Wednesday** to the consumer who prefers balanced, fruity and complex taste.

